## Setting up for curricular experiment

This assumes you have already followed the instructions in `baselines/baseline_t5`, which will set up the baseline clue files

### Datasets
1. Download and unzip the xd cw crossword set from http://xd.saul.pw/xd-clues.zip.
    - Save it as './data/original/xd/clues.tsv'
2. Preprocess the dataset using this notebook
3. The dataset will be saved to k_acw_export_dir (as a single train.json file)


In [9]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../decrypt')
sys.path.append('../')

from decrypt.scrape_parse.acw_load import get_clean_xd_clues
import config
from decrypt.common.util_data import clue_list_tuple_to_train_split_json

k_xd_orig_tsv = config.DataDirs.OriginalData.k_xd_cw
k_acw_export_dir = config.DataDirs.DataExport.xd_cw_json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# defaults to strip periods, remove questions, remove abbrevs, remove fillin
stc_map, all_clues = get_clean_xd_clues(k_xd_orig_tsv,
                                        remove_if_not_in_dict=False,
                                        do_filter_dupes=True)
clue_list_tuple_to_train_split_json((all_clues,),
                                    comment='ACW set; xd cw set, all',
                                    export_dir=k_acw_export_dir,
                                    overwrite=False)

In [5]:
# produce anagram datasets
# roughly 3 minutes to complete
from decrypt.common import anagrammer
anagrammer.gen_db_with_both_inputs(update_flag="new")

INFO:root:Reading file into dict: /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/generated/twl_dict.txt
32188it [00:00, 321873.40it/s]

Initialized a spellchecker
This will fail if you have not downloaded or generated twl_dict.txt
DEL called for spellchecker


178691it [00:00, 336759.43it/s]
INFO:root:Done reading file: /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/generated/twl_dict.txt
INFO:root:Reading file into dict: /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/original/us/US.dic
118619it [00:00, 333901.88it/s]
INFO:root:Done reading file: /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/original/us/US.dic
INFO:root:Done setting up spellchecker
INFO:root:Adding to db /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/generated/anag_db with updateflag new
190717it [03:11, 998.48it/s] 
1it [00:00,  8.15it/s]

Counter({1: 103379, 2: 49359, 3: 10649, 4: 4179, 5: 683, 6: 106, 7: 20, 8: 3, 9: 1, 10: 1})
Done.


118619it [01:05, 1799.26it/s]


Counter({1: 118609, 'dupes': 80781})
Done.
DEL called for spellchecker


In [7]:
from decrypt.common.util_data import (
    get_anags,
    write_json_tuple
)
import json

In [8]:
def make_anag_sets_json():
    all_anags = get_anags(max_num_words=-1)
    json_list = []
    for idx, a_list in enumerate(all_anags):
        json_list.append(dict(idx=idx,
                              anag_list=a_list))
    print(json_list[0])

    # normally would be (idx, input, tgt)
    output_tuple = [json_list,]

    write_json_tuple(output_tuple,
                     comment="List of all anagram groupings",
                     export_dir=config.DataDirs.DataExport.anag_dict_lists,
                     overwrite=False)

make_anag_sets_json()

INFO:root:Initializing (non-singleton) Anagrammer from /Users/jsrozner/MOUNT/scdt/cryptic_nlp/decrypt_root/data/generated/anag_db


TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

In [ ]:
# make the indicator list
with open(k_dir + "deits_solver/indicators/ana_", 'r') as f:
    all_anag_indicators = f.readlines()

In [ ]:
final_indic_list = []
for a in all_anag_indicators:
    final_indic_list.append(a.replace('_', " ").strip())
with open(DataDirs.Export.main + "json/_anag2/anag_dict_lists/indic_list.json", 'w') as f:
    json.dump(final_indic_list,f)


## Curricular training
1. At this point you should have a file at `./data/clue_json/curricular/ACW/train.json`

- training: collate functions




